In [1]:
import json
from langchain_ollama import ChatOllama

In [3]:
model_name = "llama3.2"
llama_model = ChatOllama(
            model= model_name,
            temperature=0.1,
            # other params ...
        )


https://platform.openai.com/examples

In [19]:
UTTERANCE_TYPE = "statement"
path = "/Users/piek/Desktop/d-Leolani/cltl-languagegeneration/examples/data/basic-statements-responses.json"
#path = "/Users/piek/Desktop/d-Leolani/cltl-languagegeneration/examples/data/basic-mentions-responses.json"
#UTTERANCE_TYPE = "question"
#path = "/Users/piek/Desktop/d-Leolani/cltl-languagegeneration/examples/data/basic-questions-responses.json"
file = open(path)
data = json.load(file)


In [5]:
print("Length of data", len(data))
for item in data:
    for el in item:
        print(el)
    break

Length of data 61
response
statement
thoughts
rdf_log_path


thoughts-responses is a list of responses. Each response has the following data elements:

* response: id
* statement: the statement to which a response is created with
* thoughts: the thoughts
* rdf_log_path

The statement consists of:

* chat
* turn
* author, 
* utterance, 
* subject, 
* predicate, 
* object
* perspective
* timestamp
* context_id
* triple (_subject, _predicate, _complement)

The thoughts is a list of the elements:

* _statement_novelty
* _entity_novelty
* _negation_conflicts
* _complement_conflict
* _subject_gaps
* _complement_gaps
* _overlaps
* _trust

For each thought, we need to formulate a different prompt.

In [9]:
### Get all statement from data
statements = []
for response in data:
    statements.append(response["statement"])
for item in statements[0]:
    print(item)

chat
turn
author
utterance
utterance_type
position
subject
predicate
object
perspective
timestamp
context_id
triple


In [10]:
def get_triple_text_from_statement (statement):
    triple = statement["triple"]
    triple_text = triple["_subject"]["_label"]+", "+triple["_predicate"]["_label"]+", "+triple["_complement"]["_label"]
    return triple_text

def get_perspective_from_statement  (statement):
    perspective = statement["perspective"]
    perspective_text = ""
    if not perspective['_certainty']=='UNDERSPECIFIED':
        perspective_text += perspective['_certainty']+ ", "
    if perspective['_polarity']=='POSITIVE':
        perspective_text += 'believes'+ ", "
    elif perspective['_polarity']=='NEGATIVE':
        perspective_text += 'denies'+ ", "
    if not perspective['_sentiment']=='NEUTRAL':
        perspective_text += perspective['_sentiment']+ ", "
    if not perspective['_emotion']=='UNDERSPECIFIED':
        perspective_text += perspective['_emotion']+ ", "
    return perspective_text

def get_source_from_statement (statement):
    #"author": {"label": "piek"
    author = statement["author"]["label"]
    return author

In [11]:
triple = statements[0]["triple"]
print(triple)
triple_text = triple["_subject"]["_label"]+" "+triple["_predicate"]["_label"]+" "+triple["_complement"]["_label"]
print(triple_text)
perspective = statements[0]["perspective"]
print(perspective)

{'_subject': {'_id': 'http://cltl.nl/leolani/world/lenka-1', '_label': 'lenka', '_offset': None, '_confidence': 0.0, '_types': ['person', 'Instance']}, '_predicate': {'_id': 'http://cltl.nl/leolani/n2mu/be-from', '_label': 'be-from', '_offset': None, '_confidence': 0.0, '_cardinality': 1}, '_complement': {'_id': 'http://cltl.nl/leolani/world/serbia', '_label': 'serbia', '_offset': None, '_confidence': 0.0, '_types': ['location', 'Instance']}}
lenka be-from serbia
{'_certainty': 'CERTAIN', '_polarity': 'POSITIVE', '_sentiment': 'NEUTRAL', '_time': None, '_emotion': 'UNDERSPECIFIED'}


### Instruct-1

Paraphrases triples and author perspectives expressed through a that-phrase

In [36]:
UTTERANCE_TYPE = "statement"

instruct_1_triples_perspectives = {"role": "system", "content": f"You are an intelligent assistant. \
     I will give you as input: a phrase, followed by a perspective, followed by \"that\" and a triple with a subject, predicate and object.\
     You need to paraphrase the input in plain English as a {UTTERANCE_TYPE} and express both author perspective and the triple information. \
     Only reply with the short paraphrase of the input. \
     Do not give an explanation. \
     Do not explain what the subject and object is. \
     The response should be just the paraphrased text and nothing else."
           }

In [37]:
print(path)
for statement in statements:
    triple = statement["triple"]
    triple_text = get_triple_text_from_statement(statement)
    perspective = statement["perspective"]
    perspective_text = get_perspective_from_statement(statement)
    author = get_source_from_statement(statement)
    prompt = [instruct_1_triples_perspectives, {"role": "user", "content": author + " is "+ perspective_text+ " that "+ triple_text}]
    response = llama_model.invoke(prompt)
    print()
    print('Triple text', triple_text)
    print('perspective_text', perspective_text)
    print('author', author)
    print('Generated output', response.content)

/Users/piek/Desktop/d-Leolani/cltl-languagegeneration/examples/data/basic-statements-responses.json

Triple text lenka, be-from, serbia
perspective_text CERTAIN, believes, 
author piek
Generated output Piek is certain that Lenka is from Serbia.

Triple text bram, be-from, netherlands
perspective_text CERTAIN, believes, 
author piek
Generated output Bram is from the Netherlands.

Triple text selene, be-from, mexico
perspective_text CERTAIN, believes, 
author piek
Generated output Piek is certain that Selene was born in Mexico.

Triple text suzana, be-from, croatia
perspective_text CERTAIN, believes, 
author piek
Generated output Suzana is from Croatia.

Triple text selene, be-from, netherlands
perspective_text CERTAIN, believes, 
author piek
Generated output Piek from Netherlands is certain that Selene is from there.

Triple text lenka, be-from, serbia
perspective_text PROBABLE, believes, 
author selene
Generated output Selene probably believes Lenka is from Serbia.

Triple text bram, l

# OTHER PROMPT ATTEMPTS

### Instruct-2

Simpler version to paraphrase triples and author perspectives

In [27]:
instruct_2_triples_perspectives = {"role": "system", "content": f"You are an intelligent assistant. \
     I will give you a phrase, followed by a perspective and a triple with a subject, predicate and object that you need to paraphrase in plain English as a {UTTERANCE_TYPE}. \
     Do not give an explanation, only reply with a short paraphrase of the input and spell the subject and object names as given. "}

### Instruct-3

Praphrases triples and not perspectives

In [28]:
instruct_3_triples = {"role": "system", "content": f"You are an intelligent assistant. \
     I will give you a triple with a subject, predicate and object that you need to paraphrase in plain English as a {UTTERANCE_TYPE}. \
     Do not give an explanation or comment, just paraphrase as a short statement and spell the subject and object labels exactly as given but you may capitalize them if it is a name. \
     Do not comment on the capitalization."}

### Instruct-4 subject gap

In [32]:
UTTERANCE_TYPE = "question"
instruct_4_subject_gap = {"role": "system", "content": f"You are an intelligent assistant. \
     I will give you as input: a triple with a predicate and object that lacks a subject.\
     You need to paraphrase the input in plain English as a {UTTERANCE_TYPE}. \
     Only reply with the short paraphrase of the input. \
     Do not give an explanation. \
     Do not explain what the subject and object is. \
     The response should be just the paraphrased text and nothing else."
           }

triple_text = "be-from Serbia"

prompt = [instruct_4_subject_gap, {"role": "user", "content": triple_text}]
   # prompt.append({"role": "user", "content": triple_text})
response = llama_model.invoke(prompt)
print(response.content)


Is this person from Serbia?


## Instruct-5 object gap

In [33]:
UTTERANCE_TYPE = "question"
instruct_5_object_gap = {"role": "system", "content": f"You are an intelligent assistant. \
     I will give you as input: a triple with a subject, a predicate and a type of object.\
     You need to paraphrase the input in plain English as a {UTTERANCE_TYPE} for the object. \
     Use who for the type person, where for the type location, when for the type time and what for everything else. \
     Only reply with the short paraphrase of the input. \
     Do not give an explanation. \
     Do not explain what the subject and object is. \
     The response should be just the paraphrased text and nothing else."
           }

triple_text = "Lenka be-from location"

prompt = [instruct_5_object_gap, {"role": "user", "content": triple_text}]
   # prompt.append({"role": "user", "content": triple_text})
response = llama_model.invoke(prompt)
print(response.content)



Where are you from?
